# Artificial Intelligence
## L.EIC – 3rd Year/2nd Semester
### Exercise Sheet 1
# Solving Problems by Searching

## The Two Buckets Problem

<img src="https://qph.cf2.quoracdn.net/main-qimg-45726b16b460cae0147ae8ca245a8fb0-pjlq" width="250px" height="250px" align="right">

Two buckets of capacities **c1** (e.g. 4 liters) and **c2** (e.g. 3 liters), respectively, are initially empty. The buckets do not have any intermediate markings. The only operations you can perform are:

- Fill (completely) a bucket 
- Empty a bucket.
- Pour one bucket into the other (until the target one is full or the source one is empty).

The aim is to determine which operations to carry out so that the first bucket contains exactly **n** liters (e.g. 2 litres).

Formulate this problem as a search problem by defining the state representation, initial state, operators (their name, preconditions, effects, and cost), and objective test.

General Case (capacities b1, C2).   
Operators (using moddeling 1): Emp1, Emp2, Fill1, Fill2, Pour12a, Pour12b, Pour21a, Pour21b  

| Name  | PreCond | Effects | Cost |
|-------|---------|---------|------|
| Emp1  | W1 > 0 | W1 = 0 | 1 |
| Emp2  | W2 > 0 | W2 = 0 | 1 |
| Fill1 | W1 < b1 | W1 = b1 | 1 |
| Fill2 | W2 < C2 | W2 = C2 | 1 |
| Po12a | W1 > 0 and W2 < C2 and W1 >= C2 - W2 | W1 = W1 - (C2 - W2); W2 = C2 | 1 |
| Po12b | W1 > 0 and W2 < C2 and W1 < C2 - W2 | W2 = W1 + W2; W1 = 0 | 1 |
| Po21a | W2 > 0 and W1 < b1 and W2 >= C1 - W1 | W2 = W2 - (C1 - W1); W1 = C1 | 1 |
| Po21b | W2 > 0 and W1 < C1 and W2 < C1 - W1 | W1 = W1 + W2; W2 = 0 | 1 |

Other possible Costs:  
- C2: Water Poured from tap
- C3: Water wasted
- C4: Weight carried

| Name  | PreCond | Effects | C2 | C3 |
|-------|---------|---------|----|----|
| Emp1  | W1 > 0 | W1 = 0 | 0 | W1 |
| Emp2  | W2 > 0 | W2 = 0 | 0 | W2 |
| Fill1 | W1 < C1 | W1 = C1 | C1 - W1 | 0 |
| Fill2 | W2 < C2 | W2 = C2 | C2 - W2 | 0 |
| Po12a | W1 > 0 and W2 < C2 and W1 >= C2 - W2 | W1 = W1 - (C2 - W2); W2 = C2 | 0 | 0 |
| Po12b | W1 > 0 and W2 < C2 and W1 < C2 - W2 | W2 = W1 + W2; W1 = 0 | 0 | 0 |
| Po21a | W2 > 0 and W1 < C1 and W2 >= C1 - W1 | W2 = W2 - (C1 - W1); W1 = C1 | 0 | 0 |
| Po21b | W2 > 0 and W1 < C1 and W2 < C1 - W1 | W1 = W1 + W2; W2 = 0 | 0 | 0 |

What is the size of the state space for this problem? Represent the state space by drawing the possible objective states and displaying some of the possible transitions from the initial state.

#### State Space Graph
Size = 5*4 = 20 states
|     |     |     |     |     |
|-----|-----|-----|-----|-----|
| 0,0 | 4,0 | 1,3 | 0,1 | 0,2 |
| 0,3 | 4,3 | 3,0 | 4,1 | 3,1 |
| 2,0 | 2,1 | 2,2 | 2,3 | 3,3 |
| 1,0 | 1,1 | 1,2 | 3,2 | 4,2 |


##### Nodes

- **Initial State:** 0,0
- **Emp1:** 4,0
- **Po21:** 1,3
- **Po12:** 0,2
- **Emp2:** 0,3
- **Fill2:** 4,3
- **Fill1:** 3,0
- **Po21:** 4,1
- **Po12:** 3,1
- **Po21:** 2,2
- **Po12:** 2,3
- **Solution:** 3,3
- **Solution:** 4,2
- **Impossible:** 1,0
- **Impossible:** 1,1
- **Impossible:** 1,2
- **Impossible:** 3,2

##### Edges

- **Initial State** --Emp1--> **Emp1**
- **Initial State** --Fill2--> **Emp2**
- **Emp1** --Po21--> **Po21**
- **Emp1** --Po12--> **Po12**
- **Emp2** --Fill1--> **Fill1**
- **Emp2** --Emp1--> **Fill2**
- **Fill2** --Po21--> **Po21**
- **Fill2** --Po12--> **Po12**
- **Fill1** --Po21--> **Po21**
- **Fill1** --Po12--> **Po12**
- **Po21** --Po12--> **Solution:** 3,3
- **Po12** --Po21--> **Solution:** 4,2
- **Po21** --> **Impossible:** 1,0
- **Po21** --> **Impossible:** 1,1
- **Po21** --> **Impossible:** 1,2
- **Po12** --> **Impossible:** 3,2

Solve the problem by hand, using tree search. What solutions have you found?

### Buildind a computational approach to handle the problem

To build a program to solve the buckets problem, we will implement a solution that separates the problem definition from the algorithms used to traverse the state space. This way, we can reuse our implementations of the search strategies in other problems.

#### Representing the two buckets problem as a search problem

Let's start by defining a state for the buckets problem. For that, it'll suffice to aggregate two quantities, each representing the amount of water in one of the buckets. We also define a way of printing the state.

In [26]:
c1 = 4   # capacity for bucket 1
c2 = 3   # capacity for bucket 2

class BucketState:
    c1 = 4   # capacity for bucket 1
    c2 = 3   # capacity for bucket 2
    
    def __init__(self, b1, b2):
        self.b1 = b1
        self.b2 = b2

    '''needed for the visited list'''
    def __eq__(self, other):
        if isinstance(other, self.__class__):
            return self.__dict__ == other.__dict__
        else:
            return False

    def __ne__(self, other):
        """Overrides the default implementation (unnecessary in Python 3)"""
        return not self.__eq__(other)
    
    def __hash__(self):
        return hash((self.b1, self.b2)) 
    ''' - '''

    def __str__(self):
        return "(" + str(self.b1) + ", " + str(self.b2) + ")"

Now we define each of the operators on states:

In [27]:
# Emptying the first bucket
def empty1(state):
    if state.b1 > 0:
        return BucketState(0, state.b2)
    return None

# Emptying the second bucket
def empty2(state):
    if state.b2 > 0:
        return BucketState(state.b1, 0)
    return None

# Filling the first bucket
def fill1(state):
    if state.b1 < c1:
        return BucketState(c1, state.b2)
    return None

# Filling the second bucket
def fill2(state):
    if state.b2 < c2:
        return BucketState(state.b1, c2)
    return None

# Pouring from the first bucket to the second bucket until the second bucket is full
def pour12_fill2(state):
    if state.b1 > 0 and state.b2 < c2 and state.b1 >= c2 - state.b2:
        return BucketState(state.b1 - (c2 - state.b2), c2)  
    return None

# Pouring from the first bucket to the second bucket until the first bucket is empty
def pour12_empty1(state):
    if state.b1 > 0 and state.b2 < c2 and state.b1 < c2 - state.b2:
        return BucketState(0, state.b1 + state.b2)
    return None

# Pouring from the second bucket to the first bucket until the first bucket is full
def pour21_fill1(state):
    if state.b2 > 0 and state.b1 < c1 and state.b2 >= c1 - state.b1:
        return BucketState(c1, state.b2 - (c1 - state.b1))
    return None

# Pouring from the second bucket to the first bucket until the second bucket is empty
def pour21_empty2(state):
    if state.b2 > 0 and state.b1 < c1 and state.b2 < c1 - state.b1:
        return BucketState(state.b1 + state.b2, 0)
    return None


The following function will aggregate all states that can be generated from a given one:

In [28]:
def child_bucket_states(state):
    new_states = []
    if(empty1(state)):
        new_states.append((empty1(state), "empty1"))
    if(empty2(state)):
        new_states.append((empty2(state), "empty2"))
    if(fill1(state)):
        new_states.append((fill1(state), "fill1"))
    if(fill2(state)):
        new_states.append((fill2(state), "fill2"))
    if(pour12_fill2(state)):
        new_states.append((pour12_fill2(state), "pour12_fill2"))
    if(pour12_empty1(state)):
        new_states.append((pour12_empty1(state), "pour12_empty1"))
    if(pour21_fill1(state)):
        new_states.append((pour21_fill1(state), "pour21_fill1"))
    if(pour21_empty2(state)):
        new_states.append((pour21_empty2(state), "pour21_empty2"))
    return new_states

Play around with the state transition operators and check if they are working properly:

In [29]:
s = BucketState(0, 0)
s = fill1(s)
print("fill1:")
print(s)

child_bucket_states(BucketState(0, 0))
child_states = child_bucket_states(BucketState(0, 0))
print("child_bucket_states:")
for state,action in child_states:
    print(state,action)

fill1:
(4, 0)
child_bucket_states:
(4, 0) fill1
(0, 3) fill2


Finally, we need to define the goal condition:

In [33]:
def goal_bucket_state(state_action):
    state, action = state_action
    return state.b1 == 2

Test your goal condition:

In [36]:
goal = 2
print(f"Testing goal condition for having {goal} units in bucket2:")
for state in child_states:
    if goal_bucket_state(state):
        print(f"Goal state reached: {state[0]}")
    else:
        print(f"Not a goal state: {state[0]}")

Testing goal condition for having 2 units in bucket2:
Not a goal state: (4, 0)
Not a goal state: (0, 3)


#### Implementing search algorithms

Let us start by defining an appropriate structure to represent a node in a search tree. Each tree node will include:
- a state of the problem
- a link to its parent (to allow traveling from a leaf node towards the root of the tree)
- a list of child nodes

In [37]:
# A generic definition of a tree node holding a state of the problem
class TreeNode:
    def __init__(self, state, parent=None, action=None):
        self.state = state
        self.parent = parent
        self.children = []
        self.action = action
        
    def add_child(self, child_node):
        self.children.append(child_node)
        child_node.parent = self

##### Breadth-first search

Based on this structure, we can now implement breadth-first search. Note that we want the implementation to be independent of the problem at hand (in this case, the two buckets problem).

In [42]:
from collections import deque

def breadth_first_search(initial_state, goal_state_func, operators_func):
    root = TreeNode(initial_state[0])   # create the root node in the search tree
    queue = deque([root])   # initialize the queue to store the nodes
    visited = set([initial_state[0]])   # to avoid loops – only in small problems
    while queue:
        node = queue.popleft()   # get first element in the queue
        if goal_state_func((node.state, node.action)):   # check goal state
            return node
        
        for state, action in operators_func(node.state):   # go through next states
            if state not in visited:   # to avoid loops
                # add state to visited list
                visited.add(state)
                # create tree node with the new state
                child_node = TreeNode(state=state, parent=node, action=action)
                # link child node to its parent in the tree
                node.add_child(child_node)
                # enqueue the child node
                queue.append(child_node)

    return None


We can now use this function to actually perform a breadth-first search on the buckets problem: we pass it the initial state, our goal condition function, and the function for obtaining child states.

In [45]:
goal = breadth_first_search((BucketState(0,0), None), goal_bucket_state, child_bucket_states)
print(goal.state)

(2, 3)


In order to print the actual steps from the initial state to the last, we can take advantage of each node's link to its parent.

In [48]:
def print_solution(node):
    path = []
    while node:
        path.append((node.state, node.action))
        node = node.parent
    path.reverse()
    print("Solution:")
    for state, action in path:
        print(f"State: {state}, Action: {action}")
    return

Now we can print the solution:

In [49]:
print_solution(goal)

Solution:
State: (0, 0), Action: None
State: (4, 0), Action: fill1
State: (1, 3), Action: pour12_fill2
State: (1, 0), Action: empty2
State: (0, 1), Action: pour12_empty1
State: (4, 1), Action: fill1
State: (2, 3), Action: pour12_fill2


If we need a description for each of the employed operators, we could have each operation function return also such a description, and modify the TreeNode class so that each node also includes a description of the edge to get there. We leave that as an exercise after class.

##### Depth-first search

Implement depth-first search (again, in a manner that is independent of the problem at hand). You can start from your breadth-first search implementation and with minor changes get an implementation for depth-first search.

In [50]:
def depth_first_search(initial_state, goal_state_func, operators_func):
    root = TreeNode(initial_state[0])   # create the root node in the search tree
    stack = [root]   # initialize the stack to store the nodes
    visited = set([initial_state[0]])   # to avoid loops – only in small problems
    while stack:
        node = stack.pop()   # get last element in the stack
        if goal_state_func((node.state, node.action)):   # check goal state
            return node
        
        for state, action in operators_func(node.state):   # go through next states
            if state not in visited:   # to avoid loops
                # add state to visited list
                visited.add(state)
                # create tree node with the new state
                child_node = TreeNode(state=state, parent=node, action=action)
                # link child node to its parent in the tree
                node.add_child(child_node)
                # push the child node
                stack.append(child_node)

    return None

Test it on the two buckets problem.

In [56]:
def print_solution_dfs(node):
    path = []
    while node:
        path.append((node.state, node.action))
        node = node.parent
    path.reverse()
    print("Solution:")
    for state, action in path:
        print(f"State: {state}, Action: {action}")
    print(f"Number of steps: {len(path) - 1}")
    return

print_solution_dfs(depth_first_search((BucketState(0,0), None), goal_bucket_state, child_bucket_states))

Solution:
State: (0, 0), Action: None
State: (0, 3), Action: fill2
State: (3, 0), Action: pour21_empty2
State: (3, 3), Action: fill2
State: (4, 2), Action: pour21_fill1
State: (0, 2), Action: empty1
State: (2, 0), Action: pour21_empty2
Number of steps: 6


If you are unable to get a solution, think about it: depth-first search is not a complete search method, and one of the reasons for that is if the state space contains cycles. As such, you need to make sure you avoid entering into a cycle by keeping a visited nodes list or set and checking that list whenever you generate a new state.

##### Depth-limited search

Another way to make it work is to impose a depth limit to the problem. Implement depth-limited search.

In [57]:
def depth_limited_search(initial_state, goal_state_func, operators_func, depth_limit):
    root = TreeNode(initial_state[0])   # create the root node in the search tree
    stack = [(root, 0)]   # initialize the stack to store the nodes
    visited = set([initial_state[0]])   # to avoid loops – only in small problems
    while stack:
        node, depth = stack.pop()   # get last element in the stack
        if goal_state_func((node.state, node.action)):   # check goal state
            return node
        
        if depth < depth_limit:
            for state, action in operators_func(node.state):   # go through next states
                if state not in visited:   # to avoid loops
                    # add state to visited list
                    visited.add(state)
                    # create tree node with the new state
                    child_node = TreeNode(state=state, parent=node, action=action)
                    # link child node to its parent in the tree
                    node.add_child(child_node)
                    # push the child node
                    stack.append((child_node, depth + 1))

    return None

Test it on the two buckets problem.

In [71]:
def print_solution_dls(node):
    if node is None:
        print("Solution not reached within the depth limit.")
        return
    
    path = []
    while node:
        path.append((node.state, node.action))
        node = node.parent
    path.reverse()
    
    print("Solution:")
    for state, action in path:
        print(f"State: {state}, Action: {action}")
    
    if not goal_bucket_state(path[-1]):
        print("Solution not reached within the depth limit.")
    else:
        print(f"Number of steps: {len(path) - 1}")
    return

test1 = depth_limited_search((BucketState(0,0), None), goal_bucket_state, child_bucket_states, 6)
test2 = depth_limited_search((BucketState(0,0), None), goal_bucket_state, child_bucket_states, 5)
print("Test 1:")
print_solution_dls(test1)
print("Test 2:")
print_solution_dls(test2)

Test 1:
Solution:
State: (0, 0), Action: None
State: (0, 3), Action: fill2
State: (3, 0), Action: pour21_empty2
State: (3, 3), Action: fill2
State: (4, 2), Action: pour21_fill1
State: (0, 2), Action: empty1
State: (2, 0), Action: pour21_empty2
Number of steps: 6
Test 2:
Solution not reached within the depth limit.


##### Iterative deepening search

Based on depth-limited, you can easily implement iterative-deepening search.

In [23]:
def iterative_deepening_search(initial_state, goal_state_func, operators_func, depth_limit):
    # your code here
    

Again, test it on the two buckets problem.

In [24]:
# your code here


(2, 3)
Found goal state in 6 steps:
(0, 0)
(4, 0)
(1, 3)
(1, 0)
(0, 1)
(4, 1)
(2, 3)


## The Missionaries and Cannibals Problem 

<img src="https://allfish24.files.wordpress.com/2016/09/gambar-1.jpg">

Three missionaries and three cannibals are on one of the banks of the river with a boat that only takes one or two people. The boat cannot travel the river alone.

The goal is to find a way to get the six to the other bank of the river without ever leaving more cannibals than missionaries on one of the banks (even at the instant they leave/join the boat) during the process.

Formulate this problem as a search problem by defining the state representation, initial state, operators (their name, preconditions, effects, and cost), and objective test.

Solve the problem by hand, using tree search. What solutions have you found?

Represent the problem as a search problem and take advantage of the implemented search algorithms to find solutions!

In [ ]:
# your code here
